In [1]:
import ee
import geemap

In [2]:
L8bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

In [3]:
Map = geemap.Map()
# Map

In [4]:
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    # Return the masked image, scaled to TOA reflectance, without the QA bands.
    return image.updateMask(mask).divide(10000).select(L8bands).copyProperties(image, ["system:time_start"])

In [5]:
# Creating an add variable function for Landsat 8 index calculation.
def compute_indices(image):
    NDVI = image.expression('(B5-B4)/(B5+B4)', 
                            {'B4': image.select('B4'),
                             'B5': image.select('B5')}).rename('ndvi')
    MNDWI = image.expression('(B3-B6) / (B3+B6)',
                             {'B3': image.select('B3'),
                              'B6': image.select('B6')}).rename('mndwi')
    EVI = image.expression('2.5 * ((B5 - B4)/(B5 +(6*B4)-(7.5*B2)+1))',
                           {'B2': image.select('B2'),
                            'B4': image.select('B4'),
                            'B5': image.select('B5')}).rename('evi')
    LSWI = image.expression('(B5-B6)/(B5+B6)', 
                            {'B5': image.select('B5'),
                             'B6': image.select('B6')}).rename('lswi')
    return image.addBands(NDVI).addBands(MNDWI).addBands(EVI).addBands(LSWI)